In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os
from tqdm import tqdm

In [14]:
num_epochs = 3
batch_size = 64
learning_rate = 0.01

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Using device: cuda


In [15]:
class LeNet5(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [16]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2470, 0.2435, 0.2616)),])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2470, 0.2435, 0.2616)),])

data_dir = "./data"
os.makedirs(data_dir, exist_ok=True)

train_dataset = datasets.CIFAR10(root=data_dir, train=True, download=True, transform=transform_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.CIFAR10(root=data_dir, train=False, download=True, transform=transform_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [17]:
model = LeNet5().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)
total_step = len(train_loader)

In [18]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (images, labels) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} (Training)")):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss = loss.item()

    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
           .format(epoch+1, num_epochs, i+1, total_step, running_loss))

    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in tqdm(test_loader, desc=f"Epoch {epoch+1}/{num_epochs} (Validation)"):
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs

        print('Accuracy of the network on the {} validation images: {:.2f} %'.format(5000, 100 * correct / total))

Epoch 1/3 (Training): 100%|██████████| 782/782 [00:21<00:00, 36.19it/s]


Epoch [1/3], Step [782/782], Loss: 1.5125


Epoch 1/3 (Validation): 100%|██████████| 157/157 [00:02<00:00, 67.35it/s]


Accuracy of the network on the 5000 validation images: 42.00 %


Epoch 2/3 (Training): 100%|██████████| 782/782 [00:21<00:00, 35.76it/s]


...

Accuracy of the network on the 5000 validation images: 50.69 %


Epoch 3/3 (Training): 100%|██████████| 782/782 [00:21<00:00, 36.68it/s]


Epoch [3/3], Step [782/782], Loss: 1.3524


Epoch 3/3 (Validation): 100%|██████████| 157/157 [00:02<00:00, 68.93it/s]

Accuracy of the network on the 5000 validation images: 51.27 %
